# ME5311-PROJECT:
# Climate Data Analysis: Sea Level Pressure and Two-Meter Temperature Data  Analysis

In [1]:
# Import necessary libraries
import os
import xarray as xr
from scipy.stats import pearsonr, linregress
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

dimensions of data

In [2]:
n_samples = 16071
n_latitudes = 101
n_longitudes = 161
shape = (n_samples, n_latitudes, n_longitudes)

In [3]:
path = 'data'
slp_path = os.path.join(path,'slp.nc')
t2m_path = os.path.join(path,'t2m.nc')
# load data
ds_slp = xr.open_dataset(slp_path)  # Load sea level pressure data
# print(f"ds_slp:{ds_slp}")
ds_t2m = xr.open_dataset(t2m_path)  # Load two-meter temperature data
# print(f"ds_t2m:{ds_t2m}")

ds_slp:<xarray.Dataset> Size: 1GB
Dimensions:    (time: 16071, longitude: 161, latitude: 101)
Coordinates:
  * time       (time) datetime64[ns] 129kB 1979-01-01T11:30:00 ... 2022-12-31...
  * longitude  (longitude) float32 644B 70.0 70.5 71.0 ... 149.0 149.5 150.0
  * latitude   (latitude) float32 404B 40.0 39.5 39.0 38.5 ... -9.0 -9.5 -10.0
Data variables:
    msl        (time, latitude, longitude) float32 1GB ...
Attributes:
    CDI:          Climate Data Interface version 1.9.3 (http://mpimet.mpg.de/...
    history:      Mon Feb 05 11:54:26 2024: cdo -b 32 mergetime daily_slp1979...
    Conventions:  CF-1.6
    frequency:    day
    CDO:          Climate Data Operators version 1.9.4rc1 (http://mpimet.mpg....
ds_t2m:<xarray.Dataset> Size: 1GB
Dimensions:    (time: 16071, longitude: 161, latitude: 101)
Coordinates:
  * time       (time) datetime64[ns] 129kB 1979-01-01T11:30:00 ... 2022-12-31...
  * longitude  (longitude) float32 644B 70.0 70.5 71.0 ... 149.0 149.5 150.0
  * latitude  

get data values for sea level pressure

In [4]:
da_slp_msl = ds_slp['msl']  # 'msl' is the variable name for sea level pressure
x_slp = da_slp_msl.values
# print(x_slp)
# print(x_slp.shape)

[[[101943.086 101989.92  102015.8   ... 101418.664 101386.12  101350.54 ]
  [102027.68  102279.96  102562.85  ... 101468.    101434.67  101401.91 ]
  [102285.22  102462.836 102559.64  ... 101519.43  101490.62  101461.56 ]
  ...
  [101111.01  101109.06  101107.54  ... 100526.9   100510.74  100511.875]
  [101111.2   101109.19  101108.91  ... 100658.695 100566.766 100546.74 ]
  [101112.44  101111.15  101109.95  ... 100591.01  100736.766 100558.74 ]]

 [[101900.16  101947.945 101967.85  ... 101238.516 101203.34  101165.47 ]
  [102038.18  102291.54  102573.29  ... 101279.555 101243.47  101208.89 ]
  [102372.15  102490.29  102545.21  ... 101317.945 101284.52  101252.086]
  ...
  [101124.08  101122.53  101119.734 ... 100458.69  100445.914 100452.65 ]
  [101126.08  101124.484 101118.62  ... 100604.086 100507.03  100489.336]
  [101126.445 101124.984 101117.35  ... 100552.    100682.836 100492.414]]

 [[101973.01  102020.375 102048.44  ... 101409.29  101399.12  101386.37 ]
  [102356.09  102605.1

get data values for two-meter temperature

In [5]:
da_t2m_t2m = ds_t2m['t2m']  # 't2m' is the variable name for two-meter temperature
x_t2m = da_t2m_t2m.values
print(x_t2m)
# print(x_t2m.shape)

[[[273.28217 272.29355 272.3532  ... 278.57904 278.7337  278.90683]
  [260.34695 257.77417 255.57187 ... 279.697   279.75113 279.85916]
  [267.08688 265.21936 261.89755 ... 280.44757 280.40872 280.45724]
  ...
  [299.5844  299.59753 299.64648 ... 299.56076 300.1212  300.32434]
  [299.5824  299.66254 299.65182 ... 297.05234 299.19998 299.27313]
  [299.76086 299.7094  299.669   ... 296.93024 295.269   298.79196]]

 [[274.14554 274.5057  276.38028 ... 279.28534 279.46838 279.65353]
  [261.9656  260.73248 259.91565 ... 280.26804 280.3096  280.36096]
  [266.8752  264.50937 261.25928 ... 280.97006 280.9831  280.9806 ]
  ...
  [299.7329  299.81882 299.80908 ... 299.68384 300.15683 300.06952]
  [299.7395  299.70724 299.9033  ... 296.7881  299.29776 298.99582]
  [299.72818 299.77933 300.20435 ... 296.58835 295.2061  298.81598]]

 [[274.2645  275.10245 277.218   ... 281.18222 281.30457 281.47153]
  [262.3486  260.37915 259.7292  ... 282.38742 282.32388 282.24146]
  [266.22104 264.1264  260.57538

get time snapshots from sea level pressure dataset

In [6]:
da_slp_time = ds_slp['time']
t_slp = da_slp_time.values
print(t_slp)

['1979-01-01T11:30:00.000000000' '1979-01-02T11:30:00.000000000'
 '1979-01-03T11:30:00.000000000' ... '2022-12-29T11:30:00.000000000'
 '2022-12-30T11:30:00.000000000' '2022-12-31T11:30:00.000000000']


get time snapshots from two-meter temperature dataset

In [7]:
da_t2m_time = ds_t2m['time']
t_t2m = da_t2m_time.values
print(t_t2m)

['1979-01-01T11:30:00.000000000' '1979-01-02T11:30:00.000000000'
 '1979-01-03T11:30:00.000000000' ... '2022-12-29T11:30:00.000000000'
 '2022-12-30T11:30:00.000000000' '2022-12-31T11:30:00.000000000']


get longitude values from sea level pressure dataset

In [8]:
da_slp_longitude = ds_slp['longitude']
lon_slp = da_slp_longitude.values
print(lon_slp)

[ 70.   70.5  71.   71.5  72.   72.5  73.   73.5  74.   74.5  75.   75.5
  76.   76.5  77.   77.5  78.   78.5  79.   79.5  80.   80.5  81.   81.5
  82.   82.5  83.   83.5  84.   84.5  85.   85.5  86.   86.5  87.   87.5
  88.   88.5  89.   89.5  90.   90.5  91.   91.5  92.   92.5  93.   93.5
  94.   94.5  95.   95.5  96.   96.5  97.   97.5  98.   98.5  99.   99.5
 100.  100.5 101.  101.5 102.  102.5 103.  103.5 104.  104.5 105.  105.5
 106.  106.5 107.  107.5 108.  108.5 109.  109.5 110.  110.5 111.  111.5
 112.  112.5 113.  113.5 114.  114.5 115.  115.5 116.  116.5 117.  117.5
 118.  118.5 119.  119.5 120.  120.5 121.  121.5 122.  122.5 123.  123.5
 124.  124.5 125.  125.5 126.  126.5 127.  127.5 128.  128.5 129.  129.5
 130.  130.5 131.  131.5 132.  132.5 133.  133.5 134.  134.5 135.  135.5
 136.  136.5 137.  137.5 138.  138.5 139.  139.5 140.  140.5 141.  141.5
 142.  142.5 143.  143.5 144.  144.5 145.  145.5 146.  146.5 147.  147.5
 148.  148.5 149.  149.5 150. ]


get latitude values from sea level pressure dataset

In [9]:
da_slp_latitude = ds_slp['latitude']
lat_slp = da_slp_latitude.values
print(lat_slp)

[ 40.   39.5  39.   38.5  38.   37.5  37.   36.5  36.   35.5  35.   34.5
  34.   33.5  33.   32.5  32.   31.5  31.   30.5  30.   29.5  29.   28.5
  28.   27.5  27.   26.5  26.   25.5  25.   24.5  24.   23.5  23.   22.5
  22.   21.5  21.   20.5  20.   19.5  19.   18.5  18.   17.5  17.   16.5
  16.   15.5  15.   14.5  14.   13.5  13.   12.5  12.   11.5  11.   10.5
  10.    9.5   9.    8.5   8.    7.5   7.    6.5   6.    5.5   5.    4.5
   4.    3.5   3.    2.5   2.    1.5   1.    0.5   0.   -0.5  -1.   -1.5
  -2.   -2.5  -3.   -3.5  -4.   -4.5  -5.   -5.5  -6.   -6.5  -7.   -7.5
  -8.   -8.5  -9.   -9.5 -10. ]


ONLY if not enough memory

In [10]:
# OPTIONAL: Reduce resolution if not enough memory
# Reduce resolution for sea level pressure dataset
# low_res_ds_slp = ds_slp[{'longitude': slice(None, None, 2), 'latitude': slice(None, None, 2)}]
# low_res_ds_slp.to_netcdf(path='slp_low_res.nc')
#
# # Reduce resolution for two-meter temperature dataset
# low_res_ds_t2m = ds_t2m[{'longitude': slice(None, None, 2), 'latitude': slice(None, None, 2)}]
# low_res_ds_t2m.to_netcdf(path='t2m_low_res.nc')


## Time Series Forecasting with LSTM: Sea Level Pressure and Two-Meter Temperature Data  Analysis

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

path = 'data'
slp_path = os.path.join(path, 'slp.nc')
t2m_path = os.path.join(path, 't2m.nc')

ds_slp = xr.open_dataset(slp_path)
ds_t2m = xr.open_dataset(t2m_path)

# 降低分辨率
low_res_ds_slp = ds_slp[{'longitude': slice(None, None, 2), 'latitude': slice(None, None, 2)}]
low_res_ds_slp.to_netcdf(path='slp_low_res.nc')

low_res_ds_t2m = ds_t2m[{'longitude': slice(None, None, 2), 'latitude': slice(None, None, 2)}]
low_res_ds_t2m.to_netcdf(path='t2m_low_res.nc')


ds_slp_lr = xr.open_dataset('slp_low_res.nc')
x_slp_lr = ds_slp_lr['msl'].values
print("data load")

assert x_slp.shape[0] == x_t2m.shape[0], "The number of samples must match"


x_combined = np.concatenate([x_slp, x_t2m], axis=-1)


scaler = MinMaxScaler(feature_range=(0, 1))
x_scaled = scaler.fit_transform(x_combined)

def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

# 定义LSTM模型
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, dropout_prob=0.5):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim, device=x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim, device=x.device)
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# 设置模型参数
input_dim = 1
hidden_dim = 32
num_layers = 1
output_dim = 1
dropout_prob = 0.5  # 设置 Dropout 概率
model = LSTMModel(input_dim, hidden_dim, num_layers, output_dim, dropout_prob).to(device)

# 定义损失函数和优化器
criterion = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# 训练模型
num_epochs = 5
time_step = 20

x_scaled, y_scaled = create_dataset(x_scaled, time_step)
print("create dataset complete!")

X_train, X_test, y_train, y_test = train_test_split(x_scaled, y_scaled, test_size=0.2, random_state=42)

print("train and test  dataset complete!")

X_train_tensor = torch.FloatTensor(X_train).unsqueeze(-1).to(device)
y_train_tensor = torch.FloatTensor(y_train).to(device)
X_test_tensor = torch.FloatTensor(X_test).unsqueeze(-1).to(device)
y_test_tensor = torch.FloatTensor(y_test).to(device)

batch_size = 128
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True, num_workers=8)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for i, (X_batch, y_batch) in enumerate(train_loader):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        scheduler.step()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

    if device.type == 'cuda':
        torch.cuda.empty_cache()



Using device: cuda
data load


D:\anaconda3\envs\ME5311_Project_Data_Analysis\lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


create dataset complete!
train and test  dataset complete!


D:\anaconda3\envs\ME5311_Project_Data_Analysis\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\anaconda3\envs\ME5311_Project_Data_Analysis\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/5], Loss: 0.0000
Epoch [2/5], Loss: 0.0000
Epoch [3/5], Loss: 0.0000


KeyboardInterrupt: 

In [ ]:
model.eval()
predicted_list = []
with torch.no_grad():
    for X_batch in DataLoader(X_test_tensor, batch_size=64):
        X_batch = X_batch.to(device)
        output = model(X_batch).to('cpu').numpy()
        predicted_list.append(output)

predicted = np.concatenate(predicted_list, axis=0)

predicted_rescaled = scaler.inverse_transform(predicted.reshape(-1, 1))
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))


In [ ]:
# 计算MSE和RMSE
mse = mean_squared_error(y_test_rescaled, predicted_rescaled)
rmse = np.sqrt(mse)
print(f'Mean Squared Error: {mse:.4f}')
print(f'Root Mean Squared Error: {rmse:.4f}')

# 计算R^2分数
r2 = r2_score(y_test_rescaled, predicted_rescaled)
print(f'R^2 Score: {r2:.4f}')

sampling_rate = 100
# 绘制实际值与预测值
plt.figure(figsize=(14, 7))
plt.scatter(range(0, len(y_test_rescaled), sampling_rate), y_test_rescaled[::sampling_rate], label='Actual', color='blue', s=1)
plt.scatter(range(0, len(predicted_rescaled), sampling_rate), predicted_rescaled[::sampling_rate], label='Predicted', color='red', s=1)
plt.title('Actual vs Predicted Values (Sampled)')
plt.xlabel('Sampled Number of Observations')
plt.ylabel('Rescaled Values')
plt.legend()
plt.tight_layout()
plt.show()
